# Setup to train full model with findinng proper weights to train each head (Classification + Segmentation)
## Breast-Ultrasound-Segmentation

## About Dataset
Breast cancer is one of the most common causes of death among women worldwide. Early detection helps in reducing the number of early deaths. The data reviews the medical images of breast cancer using ultrasound scan. Breast Ultrasound Dataset is categorized into three classes: normal, benign, and malignant images. Breast ultrasound images can produce great results in classification, detection, and segmentation of breast cancer when combined with machine learning.

### Data
The data collected at baseline include breast ultrasound images among women in ages between 25 and 75 years old. This data was collected in 2018. The number of patients is 600 female patients. The dataset consists of 780 images with an average image size of 500*500 pixels. The images are in PNG format. The ground truth images are presented with original images. The images are categorized into three classes, which are normal, benign, and malignant.

If you use this dataset, please cite:
Al-Dhabyani W, Gomaa M, Khaled H, Fahmy A. Dataset of breast ultrasound images. Data in Brief. 2020 Feb;28:104863. DOI: 10.1016/j.dib.2019.104863.

## Imports

In [1]:
import os

import pyrootutils

root = pyrootutils.setup_root(
    search_from=os.path.dirname(os.getcwd()),
    indicator=[".git", "pyproject.toml"],
    pythonpath=True,
    dotenv=True,
)

if os.getenv("DATA_ROOT") is None:
    os.environ["DATA_ROOT"] = f"{root}/data"

In [2]:
import torch
import torch.nn as nn

# Setup device-agnostic code
if torch.cuda.is_available():
    DEVICE = "cuda"  # NVIDIA GPU
    print("GPU Found!!")
else:
    raise Exception("No GPU Found!!")

GPU Found!!


In [3]:
import hydra
from hydra import compose, initialize

In [4]:
# # auto reload dotenv
%load_ext dotenv
%dotenv

# auto reload libs
%load_ext autoreload
%autoreload 2

## Paths setup

In [5]:
from omegaconf import DictConfig, OmegaConf

# Register a resolver for torch dtypes
OmegaConf.register_new_resolver("torch_dtype", lambda name: getattr(torch, name))

with initialize(config_path="../configs", job_name="training_setup", version_base=None):
    cfg: DictConfig = compose(config_name="train.yaml")
    # print(OmegaConf.to_yaml(cfg))
    print(cfg.models)

{'optimizer': {'_target_': 'torch.optim.Adam', '_partial_': True, 'lr': 0.001, 'weight_decay': 0.0, 'betas': [0.9, 0.999]}, 'scheduler': {'func': None}, 'model': {'_target_': 'src.models.vggnet_fcn_segmentation_model.VGGNetFCN16SegmentationModel', 'num_classes': 3}}


In [6]:
os.chdir(root)

## Loading Dataset

In [7]:
data_module = hydra.utils.instantiate(cfg.datamodule)

class_weights = data_module.class_weights
class_names = data_module.classes
num_classes = len(class_names)
class_names, num_classes, class_weights

(['normal', 'malignant', 'benign'], 3, tensor([1.9774, 1.2494, 0.5903]))

In [8]:
import torch.optim as optim
from torch.utils.data import DataLoader, Subset

# # 1. Select a very small subset of your data
train_dataset, val_dataset = data_module.split_and_preprocess_datasets()

subset_indices = list(range(100))  # Select first 10 samples
train_small_dataset = Subset(train_dataset, subset_indices)
val_small_dataset = Subset(val_dataset, subset_indices)
train_dl = DataLoader(train_small_dataset, batch_size=5, shuffle=False)
val_dl = DataLoader(val_small_dataset, batch_size=5, shuffle=False)

In [9]:
for images, targets in train_dl:
    print(images.shape, targets["masks"].shape, targets["labels"].shape)

    print(f"images:{images.dtype}, {images[0].min()}, {images[0].max()}")
    print(
        f'masks {targets["masks"].dtype}, {targets["masks"][0].min()}, {targets["masks"][0].max()}'
    )
    print(
        f'labels {targets["labels"].dtype}, {targets["labels"].min()}, {targets["labels"].max()}'
    )
    break

torch.Size([5, 3, 224, 224]) torch.Size([5, 1, 224, 224]) torch.Size([5])
images:torch.float32, -2.1179039478302, 2.640000104904175
masks torch.uint8, 0, 1
labels torch.int64, 1, 2


In [10]:
for _images, _targets in val_dl:
    print(_images.shape, _targets["masks"].shape, _targets["labels"].shape)

    print(f"images:{_images[0].dtype}, {_images[0].min()}, {_images[0].max()}")
    print(f'masks {_targets["masks"].dtype}, {_targets["masks"].min()}, {_targets["masks"].max()}')
    print(
        f'labels {_targets["labels"].dtype}, {_targets["labels"].min()}, {_targets["labels"].max()}'
    )
    break

torch.Size([5, 3, 224, 224]) torch.Size([5, 1, 224, 224]) torch.Size([5])
images:torch.float32, -2.1179039478302, 2.5354254245758057
masks torch.uint8, 0, 1
labels torch.int64, 0, 2


## Loading and training the FCN8 model 

In [11]:
from src.utils.gpu_utils import DeviceDataLoader, get_default_device, to_device

torch.cuda.empty_cache()
device = get_default_device()

gpu_weights = to_device(class_weights, device)

In [ ]:
from models.vggnet_fcn_segmentation_model import VGGNetFCN8SegmentationModel
from src.losses.dice_loss import SoftDiceLoss

model = VGGNetFCN8SegmentationModel(
    cls_num_classes=num_classes,
    seg_num_classes=1,
    vggnet_type="vgg16_bn",
    segmentation_criterion=SoftDiceLoss(),  # nn.CrossEntropyLoss(),
    classification_criterion=nn.CrossEntropyLoss(weight=gpu_weights),
)
model = torch.compile(model)
model

OptimizedModule(
  (_orig_mod): VGGNetFCN8SegmentationModel(
    (segmentation_criterion): SoftDiceLoss()
    (classification_criterion): CrossEntropyLoss()
    (cls_auroc): MulticlassAUROC()
    (encoder): VGGNetEncoder(
      (vgg): VGG(
        (features): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ReLU(inplace=True)
          (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (9): ReLU(inplace=True)
       

In [13]:
for images, labels in train_dl:
    # print(images.shape, labels)
    out = model(images)
    # l = labels['labels'][0]
    # print(l , torch.argmax(l))
    print(out["labels"][0], out["labels"].shape)
    print(out["masks"][0])
    print(out["masks"].shape)
    break

tensor([ 0.0685,  0.1413, -0.1359], grad_fn=<SelectBackward0>) torch.Size([5, 3])
tensor([[[-6.6727e-02, -5.6608e-01, -3.7048e-01,  ..., -3.3069e-01,
          -1.2427e-01,  3.5753e-03],
         [-4.9885e-02,  1.7071e-01,  6.6865e-01,  ...,  7.4345e-01,
          -1.1005e-01, -5.1679e-01],
         [-2.1225e-01,  2.0354e-01,  2.2605e-01,  ..., -1.7784e-01,
          -3.8583e-01, -2.1416e-01],
         ...,
         [-1.3516e+00, -8.6429e-01, -1.4054e+00,  ..., -2.0893e-01,
           5.0104e-01, -1.0569e-01],
         [-4.1069e-01, -1.0205e+00,  5.5551e-04,  ..., -5.1429e-01,
           1.2887e-01, -2.0566e-01],
         [-4.3268e-01,  1.2819e-01,  1.0373e-01,  ..., -8.7842e-02,
          -7.8701e-02,  5.7280e-02]]], grad_fn=<SelectBackward0>)
torch.Size([5, 1, 224, 224])


### Evaluate function

In [14]:
@torch.no_grad()
def evaluate(model, val_dl) -> dict[str, float]:
    model.eval()  # set model to evaluate mode
    outputs = [model.validation_step(batch) for batch in val_dl]
    return model.validation_epoch_end(outputs)

## GPU Training Setup

## Moving data and model into memory

In [15]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
to_device(model, device)
# train_dl.device

OptimizedModule(
  (_orig_mod): VGGNetFCN8SegmentationModel(
    (segmentation_criterion): SoftDiceLoss()
    (classification_criterion): CrossEntropyLoss()
    (cls_auroc): MulticlassAUROC()
    (encoder): VGGNetEncoder(
      (vgg): VGG(
        (features): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ReLU(inplace=True)
          (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (9): ReLU(inplace=True)
       

## Overfiting the model

In [16]:
import enum


class MetricKey(enum.Enum):
    """Keys for metrics dictionary."""

    # Classification
    CLS_LOSS = "cls_loss"
    CLS_ACC = "cls_acc"
    VAL_CLS_LOSS = "val_cls_loss"
    VAL_CLS_ACC = "val_cls_acc"

    # Segmentation
    SEG_LOSS = "seg_loss"
    SEG_DICE = "seg_dice"
    VAL_SEG_LOSS = "val_seg_loss"
    VAL_SEG_DICE = "val_seg_dice"


SEG_WEIGHT = 0.9
CLS_WEIGHT = 0.1


def fit(
    model,
    train_dataloader,
    validation_dataloader,
    epochs: int = 2,
    lr: float = 1e-3,
    opt_func=torch.optim.Adam,
) -> list[dict[str, float]]:
    history = []
    result = {}
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        masks_dice_sc = []
        train_accuracies = []
        cls_loss = []
        masks_losses = []
        for batch in train_dataloader:
            step_output = model.training_step(batch)
            seg_loss = step_output[f"{MetricKey.SEG_LOSS.value}"]
            seg_dice = step_output[f"{MetricKey.SEG_DICE.value}"]

            cls_acc = step_output[f"{MetricKey.CLS_ACC.value}"]
            cls_loss = step_output[f"{MetricKey.CLS_LOSS.value}"]

            # Detach loss and accuracy before appending to avoid holding onto computation graph
            train_losses.append(cls_loss.detach())
            masks_losses.append(seg_loss.detach())  # Assuming train_acc is a tensor
            masks_dice_sc.append(seg_dice)  # Assuming train_acc is a tensor
            train_accuracies.append(cls_acc.detach())  # Assuming train_acc is a tensor

            total_loss = (SEG_WEIGHT * seg_loss) + (CLS_WEIGHT * cls_loss)
            total_loss.backward()

            optimizer.step()
            optimizer.zero_grad()

        # Validation Phase
        result = evaluate(model, validation_dataloader)
        result[f"{MetricKey.CLS_LOSS.value}"] = torch.stack(train_losses).mean().item()
        result[f"{MetricKey.SEG_LOSS.value}"] = torch.stack(masks_losses).mean().item()
        result[f"{MetricKey.SEG_DICE.value}"] = torch.stack(masks_dice_sc).mean().item()
        result[f"{MetricKey.CLS_ACC.value}"] = torch.stack(train_accuracies).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [17]:
EPOCHS = 100
history = fit(
    model=model,
    train_dataloader=train_dl,
    validation_dataloader=val_dl,
    epochs=EPOCHS,
    lr=1e-3,
    opt_func=optim.AdamW,
)

/home/ubuntu/.local/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
/home/ubuntu/.local/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)


Epoch [0] Validation Results: cls_loss=1.0965, val_cls_loss=3.1321, cls_acc=0.3900, val_cls_acc=0.2700, val_cls_auroc=0.4042, seg_loss=0.7255, val_seg_loss=0.8098, seg_dice=0.1718, val_seg_dice=0.1605
Epoch [1] Validation Results: cls_loss=1.0741, val_cls_loss=1.2883, cls_acc=0.4800, val_cls_acc=0.3600, val_cls_auroc=0.4500, seg_loss=0.6322, val_seg_loss=0.7182, seg_dice=0.2309, val_seg_dice=0.2092
Epoch [2] Validation Results: cls_loss=1.1035, val_cls_loss=1.7989, cls_acc=0.3000, val_cls_acc=0.5100, val_cls_auroc=0.4347, seg_loss=0.6107, val_seg_loss=0.8070, seg_dice=0.2422, val_seg_dice=0.1766
Epoch [3] Validation Results: cls_loss=0.9880, val_cls_loss=2.2050, cls_acc=0.4800, val_cls_acc=0.4600, val_cls_auroc=0.6028, seg_loss=0.5537, val_seg_loss=0.8179, seg_dice=0.2755, val_seg_dice=0.1712
Epoch [4] Validation Results: cls_loss=1.1608, val_cls_loss=1.5762, cls_acc=0.4000, val_cls_acc=0.4200, val_cls_auroc=0.4458, seg_loss=0.5400, val_seg_loss=0.6532, seg_dice=0.2830, val_seg_dice=0.

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

seg_losses = [x["seg_loss"] for x in history]
seg_dice = [x["seg_dice"] for x in history]

plt.plot(seg_losses, "-bx")
plt.plot(seg_dice, "-rx")

plt.xlabel("epoch")
plt.ylabel("loss")
plt.grid()
plt.legend(["seg_loss", "seg_dice"])
plt.title("Loss vs. NO. of epochs")